In [96]:
CT_LIST = ['ESC', 'MES', 'CP', 'CM']
HM_LIST = ['H3K4me3', 'H3K27ac', 'H3K27me3',  'RNA']
PREFIXES = [HM + '_' + CT for HM in HM_LIST for CT in CT_LIST]


MARKER_GENES_EXT = {'ESC': ['Nanog','Pou5f1','Sox2','L1td1','Dppa5a','Tdh','Esrrb','Lefty1','Zfp42','Sfn','Lncenc1','Utf1'],
                    'MES': ['Mesp1','Mesp2','T', 'Vrtn','Dll3','Dll1', 'Evx1','Cxcr4','Pcdh8','Pcdh19','Robo3','Slit1'],
                    'CP':  ['Sfrp5', 'Gata5', 'Tek','Hbb-bh1','Hba-x', 'Pyy','Sox18','Lyl1','Rgs4','Igsf11','Tlx1','Ctse'],
                    'CM':  ['Nppa','Gipr', 'Actn2', 'Coro6', 'Col3a1', 'Bgn','Myh6','Myh7','Tnni3','Hspb7' ,'Igfbp7','Ndrg2'],
                    }



HM_COL_DICT = {'H3K4me3': '#f37654','H3K27ac': '#b62a77','H3K27me3': '#39A8AC','RNA':'#ED455C'}
CT_COL_DICT= {'ESC': '#405074',
                'MES': '#7d5185',
                'CP': '#c36171',
                'CM': '#eea98d',}
SET_COL_DICT= {'training':'#97DA58','validation':'#9b58da','test':'#DA5A58'}
GONZALEZ_COL_DICT= {'Active': '#E5AA44','Bivalent': '#7442BE'}

In [97]:
import plotly.express as px
import pandas as pd
import pickle



# Load gene cluster dictionary
with open(f'./data/gene_clusters_dict.pkl', 'rb') as f:
    GENE_CLUSTERS = pickle.load(f)

# Load CODE and LOG matrices
CODE = pd.read_csv(f'./data/CODE.csv', index_col='GENE')
CODE = CODE.iloc[:, 18:]
CODE.drop(columns=['AE_RMSE','AE_Sc', 'PCA_RMSE', 'PCA_Sc', 'UMAP_RMSE', 'UMAP_Sc'],inplace=True)
LOG = pd.read_csv(f'./data/ALL_X_FC.csv').set_index('GENE')


In [98]:
name = 'TOP'
N_TOP = 4000
with open(f'./data/RNA_CV/{name}{N_TOP}/dict.pkl', 'rb') as f:
    CV = pickle.load(f)
name = 'BOTTOM'
with open(f'./data/RNA_CV/{name}{N_TOP}/dict.pkl', 'rb') as f:
    BOTTOM_CV = pickle.load(f)
    
STABLE = [gene for gene_list in BOTTOM_CV.values() for gene in gene_list]
CV['STABLE'] = STABLE

CV_MAP={}
for CV_CAT, gene_list in CV.items():
    for gene in gene_list:
        CV_MAP[gene] = CV_CAT
        
CODE["CV_Category"] = CODE.index.map(CV_MAP)
CODE['CV_Category'] = CODE['CV_Category'].fillna('other')

In [99]:
with open('./data/gonzalez_dict.pkl', 'rb') as f:
    GONZALEZ = pickle.load(f)
GONZALEZ_MAP={}
for GONZALEZ_CAT, gene_list in GONZALEZ.items():
    for gene in gene_list:
        GONZALEZ_MAP[gene] = GONZALEZ_CAT
CODE["ESC_ChromState_Gonzalez2021"] = CODE.index.map(GONZALEZ_MAP)
CODE['ESC_ChromState_Gonzalez2021'] = CODE['ESC_ChromState_Gonzalez2021'].fillna('other')
CODE['ESC_ChromState_Gonzalez2021'].value_counts()

ESC_ChromState_Gonzalez2021
Active      9186
other       3495
Bivalent    2315
Name: count, dtype: int64

In [100]:

# Map cluster IDs to CODE and LOG
gene_to_cluster = {}
for cluster_id, gene_list in GENE_CLUSTERS.items():
    for gene in gene_list['gene_list']:
        gene_to_cluster[gene] = cluster_id
CODE["Cluster"] = CODE.index.map(gene_to_cluster).astype(int)


In [125]:
CODE = CODE[['RNA_ESC', 'RNA_MES', 'RNA_CP', 'RNA_CM',

    'H3K4me3_ESC','H3K4me3_MES', 'H3K4me3_CP', 'H3K4me3_CM', 'H3K27ac_ESC', 'H3K27ac_MES',
        'H3K27ac_CP', 'H3K27ac_CM', 'H3K27me3_ESC', 'H3K27me3_MES',
        'H3K27me3_CP', 'H3K27me3_CM', 
        
        'RNA_CM_CP_FC', 'RNA_CM_MES_FC', 'RNA_CM_ESC_FC', 'RNA_CP_MES_FC','RNA_CP_ESC_FC', 'RNA_MES_ESC_FC', 
        'VAE_RMSE', 'VAE_Sc', 
        'RNA_CV','CV_Category', 'ESC_ChromState_Gonzalez2021','Cluster',
        'VAE1', 'VAE2', 'VAE3', 'VAE4', 'VAE5', 'VAE6', 'VAE_UMAP1', 'VAE_UMAP2',]]
CODE['Cluster'] = pd.Categorical(CODE['Cluster'])

In [ ]:
RNA_FPKM= pd.read_csv(f'./data/RNA_FPKMs.csv', index_col='GENE')
assert list(RNA_FPKM.index) == list(CODE.index)
#concatenate the two dataframes
DATA = pd.concat([CODE, RNA_FPKM], axis=1)
DATA.to_csv(f'./data/DATA.csv')

Z_AVG_features = ['RNA_ESC', 'RNA_MES', 'RNA_CP', 'RNA_CM', 'H3K4me3_ESC', 'H3K4me3_MES',
        'H3K4me3_CP', 'H3K4me3_CM', 'H3K27ac_ESC', 'H3K27ac_MES', 'H3K27ac_CP',
        'H3K27ac_CM', 'H3K27me3_ESC', 'H3K27me3_MES', 'H3K27me3_CP',
        'H3K27me3_CM']
LOG_FC_features = ['RNA_CM_CP_FC', 'RNA_CM_MES_FC', 'RNA_CM_ESC_FC',
            'RNA_CP_MES_FC', 'RNA_CP_ESC_FC', 'RNA_MES_ESC_FC']

MISC_features = [ 'VAE_RMSE', 'VAE_Sc', 'RNA_CV', 'CV_Category', 'ESC_ChromState_Gonzalez2021', 'Cluster']

LATENT_features = ['VAE1', 'VAE2', 'VAE3', 'VAE4', 'VAE5', 'VAE6', 'VAE_UMAP1', 'VAE_UMAP2']

FPKM_features = [ 'RNA_ESC_1', 'RNA_ESC_2', 'RNA_MES_1', 'RNA_MES_2',
            'RNA_CP_1', 'RNA_CP_2', 'RNA_CM_1', 'RNA_CM_2']


Z_AVG = DATA[Z_AVG_features]

LOG_FC = DATA[LOG_FC_features]

MISC = DATA[MISC_features]

LATENT = DATA[LATENT_features]

FPKM = DATA[FPKM_features]


In [129]:
point_size = 3
colormap = "Spectral_r"
def scatter_continous(DATA, selected_feature, point_size=3, colormap='Spectral_r'):
    fig = px.scatter(
        DATA,
        x="VAE_UMAP1",
        y="VAE_UMAP2",
        color=selected_feature,
        hover_data=[DATA.index,"Cluster", selected_feature],
        title=selected_feature,
        labels={"VAE_UMAP1": "UMAP1" , "VAE_UMAP2": "UMAP2"},
        color_continuous_scale=colormap
    )
    fig.update_traces(marker=dict(size=point_size))
    fig.update_layout(
        xaxis_showgrid=False, yaxis_showgrid=False,
        xaxis_tickvals=[], yaxis_tickvals=[],
        plot_bgcolor="white", autosize=True
    )
    return fig

In [132]:
fig = scatter_continous(DATA, 'RNA_ESC')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'customdata': array([['0610005C13Rik', '20', -0.5624739402515824],
                                   ['0610010F05Rik', '71', -0.4274550058627567],
                                   ['0610010K14Rik', '2', 1.3808814616144478],
                                   ...,
                                   ['Zzef1', '4', -0.2412975110979243],
                                   ['Zzz3', '4', 0.2704404635897117],
                                   ['a', '42', -1.407244944298753]], dtype=object),
              'hovertemplate': ('UMAP1=%{x}<br>UMAP2=%{y}<br>GE' ... '%{marker.color}<extra></extra>'),
              'legendgroup': '',
              'marker': {'color': array([-0.56247394, -0.42745501,  1.38088146, ..., -0.24129751,  0.27044046,
                                         -1.40724494]),
                         'coloraxis': 'coloraxis',
                         'size': 3,
                         'symbol': 'circle'},
              'mode': 'markers',
              'name': '',
              'showlegend': False,
              'type': 'scattergl',
              'x': array([-1.4199568 ,  5.4965925 , 10.083457  , ...,  8.17686   , 10.252608  ,
                           0.13952795]),
              'xaxis': 'x',
              'y': array([ 6.77156  , 10.1938095,  6.8323727, ..., 11.855313 , 10.654977 ,
                           6.093042 ]),
              'yaxis': 'y'}],
    'layout': {'autosize': True,
               'coloraxis': {'colorbar': {'title': {'text': 'RNA_ESC'}},
                             'colorscale': [[0.0, 'rgb(94,79,162)'], [0.1,
                                            'rgb(50,136,189)'], [0.2,
                                            'rgb(102,194,165)'], [0.3,
                                            'rgb(171,221,164)'], [0.4,
                                            'rgb(230,245,152)'], [0.5,
                                            'rgb(255,255,191)'], [0.6,
                                            'rgb(254,224,139)'], [0.7,
                                            'rgb(253,174,97)'], [0.8,
                                            'rgb(244,109,67)'], [0.9,
                                            'rgb(213,62,79)'], [1.0,
                                            'rgb(158,1,66)']]},
               'legend': {'tracegroupgap': 0},
               'plot_bgcolor': 'white',
               'template': '...',
               'title': {'text': 'RNA_ESC'},
               'xaxis': {'anchor': 'y',
                         'domain': [0.0, 1.0],
                         'showgrid': False,
                         'tickvals': [],
                         'title': {'text': 'UMAP1'}},
               'yaxis': {'anchor': 'x',
                         'domain': [0.0, 1.0],
                         'showgrid': False,
                         'tickvals': [],
                         'title': {'text': 'UMAP2'}}}
})